Finding the total number of enteries in the dataset:
    

In [2]:
import gzip
count=0
for i in range(0,500):
    num=str(i)
    zero_fill_num=num.zfill(5)
    with gzip.open('/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_events/part-'+zero_fill_num+'-of-00500.csv.gz', 'rb') as f:
        for line in f:
            line=line.decode()
            line=line.split(',')
            t=line[0]
            count+=1
print(count)


144648288


Total number of enteries in the files are :144648288

Average entry per file :289296.5

Data attributes in the file :8

## Getting the file size of the dataset

In [2]:
from pathlib import Path
import gzip
for i in range(0,500):
    num=str(i)
    zero_fill_num=num.zfill(5)
    size+=Path('/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_events/part-'+zero_fill_num+'-of-00500.csv.gz').stat().st_size
print(size)

1624496362


Total file size of the Zipped file :1624496362 byte or 1.62 GB

## Reading the dataset using spark since the data size is big :

In [1]:
import os
def setup_my_environment():
    import os
    
def setenv(var, val):
    os.environ[var] = val

def prepend_path(var, val):
    old_val = os.environ.get(var, '')
    os.environ[var] = val + ":" + old_val
    
def setup_java():
    PKG_ROOT='/ichec/packages/java/8'
    setenv('JAVA_PATH', PKG_ROOT)
    setenv('JAVA_HOME', PKG_ROOT)
    prepend_path('PATH', PKG_ROOT + '/bin')
    prepend_path('MANPATH', PKG_ROOT + '/man')
    prepend_path('CPATH', PKG_ROOT + '/include')
    
def setup_spark():
    PKG_ROOT='/ichec/packages/spark/2.3.3/kay/spark-2.3.3'
    setenv('SPARK_DIST_CLASSPATH', PKG_ROOT + 'spark-2.3.3-bin-kay-spark')
    prepend_path('PATH', PKG_ROOT + PKG_ROOT + 'spark-2.3.3-bin-kay-spark/bin')
setup_java()
setup_spark()
setup_my_environment()
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

 

spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.instances","20")\
    .config("spark.executor.cores","2")\
    .config("spark.worker.instances","20")\
    .config("spark.memory.offHeap.enabled","true")\
    .config("spark.memory.offHeap.size","25g")\
    .config("spark.debug.maxToStringsFields","100")\
    .appName("GoogleSvrLog") \
    .getOrCreate()

In [2]:
df = spark.read.csv("/ichec/work/mucom001c/GoogleSvrLog/clusterdata-2011-1/task_events/*.gz")
df5=df.select('_c0','_c2','_c3','_c5','_c8','_c9','_c10','_c11')
df5=df5.selectExpr('_c0 as time','_c2 as job_id','_c3 as task_index','_c5 as event_type','_c8 as priority','_c9 as cpu_request','_c10 as memory_request','_c11 as disk_space')

In [3]:
df5.show(5)

+------------+----------+----------+----------+--------+-----------+--------------+----------+
|        time|    job_id|task_index|event_type|priority|cpu_request|memory_request|disk_space|
+------------+----------+----------+----------+--------+-----------+--------------+----------+
|807403811171|6324858588|      8323|         3|       0|     0.0125|        0.0159| 0.0004044|
|807403811182|6324858588|      8323|         0|       0|     0.0125|        0.0159| 0.0004044|
|807403811183|6324858588|      9024|         3|       0|     0.0125|        0.0159| 0.0004044|
|807403811194|6324858588|      9024|         0|       0|     0.0125|        0.0159| 0.0004044|
|807403811195|6324858588|      7152|         3|       0|     0.0125|        0.0159| 0.0004044|
+------------+----------+----------+----------+--------+-----------+--------------+----------+
only showing top 5 rows



## converting the time unix time stamp to the date in which the task was run

In [4]:
from pyspark.sql.functions import *

df5new = df5.withColumn('time', from_unixtime(df5.time, "dd"))

In [5]:
df5new.show()


+----+----------+----------+----------+--------+-----------+--------------+----------+
|time|    job_id|task_index|event_type|priority|cpu_request|memory_request|disk_space|
+----+----------+----------+----------+--------+-----------+--------------+----------+
|  11|6324858588|      8323|         3|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|      8323|         0|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|      9024|         3|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|      9024|         0|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|      7152|         3|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|      7152|         0|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|     13525|         3|       0|     0.0125|        0.0159| 0.0004044|
|  11|6324858588|     13525|         0|       0|     0.0125|        0.0159| 0.0004044|
|  11|6325302687|      2547|         4|    

In [6]:
df5new.select('time').distinct().show(35)

+----+
|time|
+----+
|  07|
|  15|
|  11|
|  29|
|  30|
|  01|
|  22|
|  28|
|  16|
|null|
|  31|
|  18|
|  27|
|  17|
|  26|
|  09|
|  05|
|  19|
|  23|
|  08|
|  03|
|  25|
|  02|
|  06|
|  24|
|  20|
|  10|
|  12|
|  04|
|  13|
|  21|
|  14|
+----+



In [7]:
df5new=df5new.groupBy('time','event_type','disk_space',"cpu_request",'memory_request').count()

In [8]:
df5new.count()

761066

## viewing the time of the given data 

In [9]:
df5new.show(10)

+----+----------+----------+-----------+--------------+------+
|time|event_type|disk_space|cpu_request|memory_request| count|
+----+----------+----------+-----------+--------------+------+
|  11|         0| 0.0001154|      0.125|       0.04773|  2306|
|  06|         2| 0.0003862|     0.0625|       0.07959|    13|
|  12|         2| 0.0001154|    0.06873|       0.01193|  9944|
|  29|         1| 5.436e-05|      0.125|       0.07959|  2404|
|  24|         2| 0.0001154|      0.125|       0.04773|   109|
|  25|         0| 0.0004044|     0.0125|        0.0159|400619|
|  25|         0|  0.001131|    0.03125|       0.07764|   221|
|  09|         5|  2.67e-05|    0.01874|      0.003109|  1081|
|  20|         0|         0|   0.003124|     6.199e-05|    45|
|  14|         1|  0.001131|    0.03125|       0.07764|   287|
+----+----------+----------+-----------+--------------+------+
only showing top 10 rows



In [13]:
df5new=df5new.repartition(1)

In [14]:
df5new.write.csv("/ichec/home/users/nishantk2106/final_dataset.csv","overwrite")